Implement a matrix factorisation using gradient descent

In [0]:
import torch
from typing import Tuple
def sgd_factorise(A:torch.Tensor, rank:int, numepochs=1000,lr=0.01):
  U = torch.randn(A.shape[0],rank)
  V = torch.randn(A.shape[1],rank)
  for epoch in range(numepochs):
    for r in range(A.shape[0]):
      for c in range(A.shape[1]):
        e = A[r][c] - U[r]@(V[c].t())
        U[r] = U[r] + e*lr*V[c]
        V[c] = V[c] + e*lr*U[r]
  return U,V

#A = torch.randn(8,7)
#rank = 4
#B,C = sgd_factorise(A,rank)
#print(A,A.shape)
#print(B,B.shape)
#print(C,C.shape)

Factorise and compute reconstruction error

In [4]:
A = torch.Tensor([[0.3374,0.6005,0.1735],
[3.3359,0.0492,1.8374],
[2.9407,0.5301,2.2620]])
rank=2
B,C = sgd_factorise(A,rank)
print(B)
print(C)
def reconstruct_loss(B:torch.Tensor,C:torch.tensor,A:torch.Tensor):
  loss = torch.nn.functional.mse_loss(B@C.t(),A,reduction='sum')
  return loss
loss = reconstruct_loss(B,C,A)
print(loss)

tensor([[-0.0987,  0.4110],
        [ 1.7130,  0.2237],
        [ 1.3378,  0.6856]])
tensor([[ 1.7758,  0.9580],
        [-0.1637,  1.1970],
        [ 1.0004,  1.1304]])
tensor(0.1222)


Compare to the truncated-SVD

In [0]:
u, s, v = torch.svd(A)
print(s)
s[len(s)-1]=0
diag_s = torch.empty((u.shape[1],u.shape[1]))
torch.nn.init.zeros_(diag_s)
for i in range(len(s)):
  diag_s[i][i]=s[i]
print(u)
print(diag_s)
print(v)
recon = u@diag_s@v.t()
print(recon)
print(A)
print(torch.nn.functional.mse_loss(recon,A,reduction='sum'))

tensor([5.3339, 0.6959, 0.3492])
tensor([[-0.0801, -0.7448,  0.6625],
        [-0.7103,  0.5090,  0.4863],
        [-0.6994, -0.4316, -0.5697]])
tensor([[5.3339, 0.0000, 0.0000],
        [0.0000, 0.6959, 0.0000],
        [0.0000, 0.0000, 0.0000]])
tensor([[-0.8349,  0.2548,  0.4879],
        [-0.0851, -0.9355,  0.3430],
        [-0.5439, -0.2448, -0.8027]])
tensor([[ 0.2245,  0.5212,  0.3592],
        [ 3.2530, -0.0090,  1.9737],
        [ 3.0378,  0.5983,  2.1023]])
tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])
tensor(0.1219)


What is the error? How does it compare to the result from your algorithm above?Why is this the case?

Ans: the error is 0.1219. This result is similar to my algorithm, even a little bit better. It's because when we approximate the matrix, the bigger eigenvalues and eignevectors will decide the main feature of the matrix, the last eigenvalue has the minimal impact. So we can reconstruct the matrix by low-rank approximatation without losing much information.

In [0]:
def sgd_factorise_masked(A:torch.Tensor, M:torch.tensor, rank:int, numepochs=1000,lr=0.01):
  U = torch.randn(A.shape[0],rank)
  V = torch.randn(A.shape[1],rank)
  for epoch in range(numepochs):
    for r in range(A.shape[0]):
      for c in range(A.shape[1]):
        if A[r][c] != 0:
          e = A[r][c] - U[r]@(V[c].t())
          U[r] = U[r] + e*lr*V[c]
          V[c] = V[c] + e*lr*U[r]
  return U,V

In [0]:
A_in = torch.Tensor([[0.3374,0.6005,0.1735],
[3.3359,0.0492,1.8374],
[2.9407,0.5301,2.2620]])
M = torch.Tensor([[1,1,1],
[0,1,1],
[1,0,1]])
U_in , V_in = sgd_factorise_masked(A_in,M,rank)
A_in_re = U_in@V_in.t()
print(A_in_re)
print(A)
print(torch.nn.functional.mse_loss(A_in_re,A,reduction='sum'))

tensor([[ 0.2211,  0.5147,  0.3621],
        [ 3.2534, -0.0117,  1.9714],
        [ 3.0348,  0.5997,  2.1091]])
tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])
tensor(0.1220)
